### `Устанавливаем необходимые зависимости`

In [1]:
! pip3 install -q pyspark pyarrow parquet-tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.9 MB/s eta 0:00:00


### `Готовим SparkContext`

Как следует из лекции объект SparkContext является точкой входа для работы со Spark кластером.

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

# Создаём конфигурационный класс с параметрами подключения
conf = (
    SparkConf()
        # Указываем URL master ноды Spark кластера
        # Можно использовать local mode, указав `local[<number_cores>]`
        # В таком случае вся обработка будет происходить на текущем компьютере
        # При этом, это может давать преимущество ввиду наличия параллелизма по ядрам компьютера
        .setMaster('local[*]')
)
# Создаём точку доступа на кластер. Позволят использовать RDD API
sc = SparkContext(conf=conf)
# Точка доступа для использования DataFrame API
spark = SparkSession(sc)

# По завершении программы нужно обязательно выполнить остановку подключения для освобождения занятых ресурсов
# sc.stop()

### `Загрузка данных`

Атрибуты датасета **calendar.parquet**.

Датасет содержит календарные данные, которые помогают связать продажи товаров с конкретными днями, неделями и событиями.

1. **date**: Дата записи данных.

2. **wm_yr_wk**: Календарная неделя в формате года и номера недели (год-неделя).

3. **weekday**: Название дня недели.

4. **wday**: Порядковый номер дня недели.

5. **month**: Порядковый номер месяца.

6. **year**: Год наблюдения.

7. **d**: Идентификатор дня в формате последовательности (например, d_1, d_2 и т.д.).

8. **event_name_1**: Название первичного события (праздники, акции, особые события).

9. **event_type_1**: Тип первичного события (например, праздник, спортивное событие).

10. **event_name_2**: Название вторичного события (если в этот день несколько значимых событий).

11. **event_type_2**: Тип вторичного события.

12. **snap_CA, snap_TX, snap_WI**: Индикаторы (0 или 1) программы SNAP (льготная покупка продуктов) в соответствующих штатах (Калифорния, Техас, Висконсин). Показатель 1 означает, что в указанный день были доступны льготы SNAP.

In [3]:
# Считаем файл calendar.parquet и запишем данные в DataFrame с названием df_calendar
df_calendar = spark.read.parquet("calendar.parquet")
df_calendar.show(5)

+----------+--------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+
|      date|wm_yr_wk|  weekday|wday|month|year|  d|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|
+----------+--------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+
|2011-01-29|   11101| Saturday|   1|    1|2011|d_1|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-01-30|   11101|   Sunday|   2|    1|2011|d_2|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-01-31|   11101|   Monday|   3|    1|2011|d_3|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|
|2011-02-01|   11101|  Tuesday|   4|    2|2011|d_4|        NULL|        NULL|        NULL|        NULL|      1|      1|      0|
|2011-02-02|   11101|Wednesday|   5|    2|2011|d_5|        NULL|        NULL|        NULL|        NULL| 

Атрибуты датасета **sales.parquet**.

Данный датасет содержит историю продаж товаров в розничных магазинах Walmart. Используется для анализа спроса, прогнозирования продаж и оптимизации запасов.

1. **id**: Уникальный идентификатор товара в конкретном магазине.

2. **item_id**: Идентификатор товара.

3. **dept_id**: Идентификатор отдела, к которому относится товар.

4. **cat_id**: Категория товара.

5. **store_id**: Идентификатор магазина, в котором был продан товар.

6. **state_id**: Штат, в котором расположен магазин.

7. **d_1, d_2, ..., d_n**: Ежедневные данные о продажах данного товара в указанном магазине (количество проданных единиц за каждый день). Каждый атрибут соответствует одному дню, начиная с первого дня периода наблюдения.

In [4]:
# Считаем файл sales.parquet и запишем данные в DataFrame с названием df_sales
df_sales = spark.read.parquet("sales.parquet")
df_sales.show(5)

+--------------------+-------------+---------+-------+--------+--------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----

Атрибуты датасета **prices.parquet**.

Датасет содержит информацию о динамике цен на товары в различных магазинах за определенные недели.

1. **store_id** – идентификатор магазина, в котором продаётся товар.

2. **item_id** – уникальный идентификатор конкретного товара.

3. **wm_yr_wk** – календарная неделя года, к которой относится указанная цена (в формате Walmart-календаря).

4. **sell_price** – розничная цена продажи товара в указанном магазине в течение соответствующей недели.

In [5]:
# Считаем файл prices.parquet и запишем данные в DataFrame с названием df_prices
df_prices = ... # ВАШ КОД
df_prices.show(5)

+--------+-------------+--------+----------+
|store_id|      item_id|wm_yr_wk|sell_price|
+--------+-------------+--------+----------+
|    CA_1|HOBBIES_1_001|   11325|      9.58|
|    CA_1|HOBBIES_1_001|   11326|      9.58|
|    CA_1|HOBBIES_1_001|   11327|      8.26|
|    CA_1|HOBBIES_1_001|   11328|      8.26|
|    CA_1|HOBBIES_1_001|   11329|      8.26|
+--------+-------------+--------+----------+
only showing top 5 rows



### `Задачи`

In [6]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [7]:
# 1. Создайте новую колонку 'is_weekend' в df_calendar, которая показывает выходной день (Saturday or Saturday - 1, в ином случае 0).
# Используйте только DataFrame API и функции, импортированные из F.
# Подсказка: Используйте функции: when, isin.
df_calendar = ... # ВАШ КОД
df_calendar.show(5)

+----------+--------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+----------+
|      date|wm_yr_wk|  weekday|wday|month|year|  d|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|is_weekend|
+----------+--------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+----------+
|2011-01-29|   11101| Saturday|   1|    1|2011|d_1|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         1|
|2011-01-30|   11101|   Sunday|   2|    1|2011|d_2|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         1|
|2011-01-31|   11101|   Monday|   3|    1|2011|d_3|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         0|
|2011-02-01|   11101|  Tuesday|   4|    2|2011|d_4|        NULL|        NULL|        NULL|        NULL|      1|      1|      0|         0|
|2011-02-02|   11101|Wednes

In [8]:
# 2. Переименуйте колонку 'wm_yr_wk' в 'week_id' в df_calendar
df_calendar = df_calendar... # ВАШ КОД
df_calendar.show(5)

+----------+-------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+----------+
|      date|week_id|  weekday|wday|month|year|  d|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|is_weekend|
+----------+-------+---------+----+-----+----+---+------------+------------+------------+------------+-------+-------+-------+----------+
|2011-01-29|  11101| Saturday|   1|    1|2011|d_1|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         1|
|2011-01-30|  11101|   Sunday|   2|    1|2011|d_2|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         1|
|2011-01-31|  11101|   Monday|   3|    1|2011|d_3|        NULL|        NULL|        NULL|        NULL|      0|      0|      0|         0|
|2011-02-01|  11101|  Tuesday|   4|    2|2011|d_4|        NULL|        NULL|        NULL|        NULL|      1|      1|      0|         0|
|2011-02-02|  11101|Wednesday|   5

In [9]:
# 3. Выведите уникальные значения weekday в df_calendar
... # ВАШ КОД

+---------+
|  weekday|
+---------+
|Wednesday|
|  Tuesday|
|   Friday|
| Thursday|
| Saturday|
|   Monday|
|   Sunday|
+---------+



In [10]:
# 4. Выведите количестко уникальных магазинов в df_sales
# Подсказка: используйте атрибут 'store_id'
...count() # ВАШ КОД

10

In [11]:
# 5. Выведите среднее количество продаж по категориям товаров за d_1
# Подсказка: используйте датасет df_sales
... # ВАШ КОД

+---------+------------------+
|   cat_id|         avg_sales|
+---------+------------------+
|    FOODS|1.6129436325678497|
|HOUSEHOLD|0.5433619866284622|
|  HOBBIES|0.6661946902654867|
+---------+------------------+



In [12]:
# 6. Создайте новый атрибут 'sell_price_int' в df_prices, изменив тип данных атрибута 'sell_price' на Integer
# Подсказка, для изменения типа используйте метод cast
df_prices = df_prices... # ВАШ КОД
df_prices.show(5)

+--------+-------------+--------+----------+--------------+
|store_id|      item_id|wm_yr_wk|sell_price|sell_price_int|
+--------+-------------+--------+----------+--------------+
|    CA_1|HOBBIES_1_001|   11325|      9.58|             9|
|    CA_1|HOBBIES_1_001|   11326|      9.58|             9|
|    CA_1|HOBBIES_1_001|   11327|      8.26|             8|
|    CA_1|HOBBIES_1_001|   11328|      8.26|             8|
|    CA_1|HOBBIES_1_001|   11329|      8.26|             8|
+--------+-------------+--------+----------+--------------+
only showing top 5 rows



In [13]:
# 7. Выведите 5 самых дорогих товаров в df_prices
... # ВАШ КОД

+--------+---------------+--------+----------+--------------+
|store_id|        item_id|wm_yr_wk|sell_price|sell_price_int|
+--------+---------------+--------+----------+--------------+
|    WI_3|HOUSEHOLD_2_406|   11317|    107.32|           107|
|    WI_3|HOUSEHOLD_2_406|   11318|    107.32|           107|
|    WI_3|HOUSEHOLD_2_406|   11319|    107.32|           107|
|    WI_2|HOUSEHOLD_2_406|   11242|     61.46|            61|
|    WI_2|HOUSEHOLD_2_406|   11247|     61.46|            61|
+--------+---------------+--------+----------+--------------+
only showing top 5 rows



In [14]:
# 8. Используя sql, найдите среднее количество продаж по штатам (назвав атрибут 'avg_sales')
df_sales.createOrReplaceTempView("sales")
spark.sql("...").show(5) # ВАШ КОД

+--------+------------------+
|state_id|         avg_sales|
+--------+------------------+
|      CA|1.1639061987536898|
|      TX|1.0318137094129223|
|      WI|0.9837105061768886|
+--------+------------------+



In [15]:
# 9. Выведите 10 строк датасета с товарами, проданных в Техасе
# Подсказка: используйте датасет df_sales
... # ВАШ КОД

+-----------------------------+-------------+---------+-------+--------+--------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-

In [16]:
# 10. Выведите 5 самых высоких цен на товары (sell_price), которые были в период с "2016-01-01" по "2016-01-31"
# Подсказка: используйте join, используйте метод between, используйте метод distinct() для уникальности цен
(
... # ВАШ КОД
)

+----------+
|sell_price|
+----------+
|     29.97|
|     29.96|
|     28.96|
|     27.98|
|     26.98|
+----------+
only showing top 5 rows



In [17]:
# 11. Найдите товары с наибольшими продажами в каждой категории (cat_id) за d_1, используя датасет df_sales
# Подсказка: используйте оконную функцию rank()
window_spec = Window... # ВАШ КОД

df_sales... # ВАШ КОД

+--------------------+---------------+-----------+---------+--------+--------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----

In [18]:
# 12. Создайте новую колонку "price_category" в df_prices, указав высокая или низкая цена: если цена больше 5, то "High", иначе "Low"
# Подсказка: используйте конструкцию when().otherwise
... # ВАШ КОД

+--------+-------------+--------+----------+--------------+--------------+
|store_id|      item_id|wm_yr_wk|sell_price|sell_price_int|price_category|
+--------+-------------+--------+----------+--------------+--------------+
|    CA_1|HOBBIES_1_001|   11325|      9.58|             9|          High|
|    CA_1|HOBBIES_1_001|   11326|      9.58|             9|          High|
|    CA_1|HOBBIES_1_001|   11327|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11328|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11329|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11330|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11331|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11332|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11333|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001|   11334|      8.26|             8|          High|
|    CA_1|HOBBIES_1_001| 

In [19]:
# 13. Вычислите суммарные продажи по всем магазинам (store_id) за d_1
... # ВАШ КОД

+--------+-----------+
|store_id|total_sales|
+--------+-----------+
|    TX_2|       3852|
|    TX_1|       2556|
|    CA_4|       1625|
|    CA_2|       3494|
|    CA_1|       4337|
|    CA_3|       4739|
|    WI_2|       2256|
|    WI_3|       4038|
|    WI_1|       2704|
|    TX_3|       3030|
+--------+-----------+

